In [ ]:
pip install datasets

In [ ]:
pip install stanza

In [ ]:
pip install allennlp allennlp_models

In [ ]:
!pip install accelerate

In [ ]:
import stanza
#stanza.download_corenlp(dir='path/to/save/corenlp')
stanza.download('en')
nlp = stanza.Pipeline('en')
doc = nlp("Barack Obama was born in Hawaii.")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
print(doc.entities)

[{
  "text": "Barack Obama",
  "type": "PERSON",
  "start_char": 0,
  "end_char": 12
}, {
  "text": "Hawaii",
  "type": "GPE",
  "start_char": 25,
  "end_char": 31
}]


In [ ]:
import numpy
import pandas
import torch
import transformers

In [ ]:
from datasets import load_dataset

dataset = load_dataset("spider", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})

In [ ]:
dataset['query']

In [ ]:
def parse_with_stanford(parser, dataset):
    results = []
    for example in dataset:
        parsed_output = parser(example['question'])
        results.append({
            'input': example['question'],
            'predicted': parsed_output,
            'gold_standard': example['query']
        })
    return results


In [ ]:
x = parse_with_stanford(nlp, dataset)

In [ ]:
import json
file_path = 'parsed_results.json'

# Write the results to a JSON file
with open(file_path, 'w') as f:
    json.dump(x, f)

print("Results have been saved to", file_path)

In [ ]:
print(x[:5])

[{'input': 'How many heads of the departments are older than 56 ?', 'predicted': [
  [
    {
      "id": 1,
      "text": "How",
      "lemma": "how",
      "upos": "ADV",
      "xpos": "WRB",
      "feats": "PronType=Int",
      "head": 2,
      "deprel": "advmod",
      "start_char": 0,
      "end_char": 3,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "many",
      "lemma": "many",
      "upos": "ADJ",
      "xpos": "JJ",
      "feats": "Degree=Pos",
      "head": 3,
      "deprel": "amod",
      "start_char": 4,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "heads",
      "lemma": "head",
      "upos": "NOUN",
      "xpos": "NNS",
      "feats": "Number=Plur",
      "head": 8,
      "deprel": "nsubj",
      "start_char": 9,
      "end_char": 14,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "of",
  

In [ ]:
with open('file.txt', 'w') as f:
    data = []
    for line in x:
        f.write(line + '\n')
        print(line)

In [ ]:
import pickle
with open("parsePickle", "wb") as fp:
  pickle.dump(x, fp)

In [ ]:
with open("parsePickle", "rb") as fp:   # Unpickling
   b = pickle.load(fp)

In [ ]:
# Example text parsing
doc = nlp("List the number of employees in each department.")
for sentence in doc.sentences:
    for word in sentence.words:
        print(f"word: {word.text}, parent: {sentence.words[word.head-1].text if word.head > 0 else 'root'}, relation: {word.deprel}")


word: List, parent: root, relation: root
word: the, parent: number, relation: det
word: number, parent: List, relation: obj
word: of, parent: employees, relation: case
word: employees, parent: number, relation: nmod
word: in, parent: department, relation: case
word: each, parent: department, relation: det
word: department, parent: employees, relation: nmod
word: ., parent: List, relation: punct


In [ ]:
def stanza_nlp_to_sql(doc):
    select_clause = []
    from_clause = []
    where_clause = []
    conditions = []

    # Simplified example to identify components of SQL
    for sentence in doc.sentences:
        for word in sentence.words:
            # Simple rule to identify potential SELECT attributes
            if word.upos == "NOUN" and word.xpos in ["NN", "NNS"]:
                if word.text.lower() not in ["number", "count", "total"]:  # avoiding aggregate functions for simplicity
                    select_clause.append(word.text)

            # Detecting potential FROM clause values
            if word.deprel == "nmod" and word.upos == "PROPN":
                from_clause.append(word.text)

            # Building WHERE clause based on adjectives or numeric modifiers
            if word.deprel in ["amod", "nummod"] and word.head > 0:
                head_word = sentence.words[word.head - 1].text
                conditions.append(f"{head_word} = '{word.text}'")

    # Constructing clauses from identified parts
    if not select_clause:  # default select clause if none identified
        select_clause.append("*")
    select_part = "SELECT " + ", ".join(select_clause)
    from_part = "FROM " + ", ".join(from_clause) if from_clause else "FROM [TABLE]"  # Placeholder if no table identified
    where_part = "WHERE " + " AND ".join(conditions) if conditions else ""

    # Combine parts to form final SQL query
    sql_query = f"{select_part} {from_part} {where_part}"
    return sql_query.strip()


# Example use
doc = nlp("How many heads of the departments are older than 56 ?")
sql_query = stanza_nlp_to_sql(doc)
print(sql_query)


SELECT heads, departments FROM [TABLE] WHERE heads = 'many'


In [ ]:
import spacy

# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def spacy_nlp_to_sql(doc):
    select_clause = []
    from_clause = []
    where_clause = []
    conditions = []

    for token in doc:
        if token.dep_ == 'attr' or token.dep_ == 'dobj':
            # Assuming attribute directly follows "What is/are"
            select_clause = token.text
        elif token.dep_ == 'prep' and token.text.lower() == 'of':
            # Assuming entity follows the preposition "of"
            if token.head.dep_ in ('attr', 'dobj') and token.nbor().pos_ != 'DET':
                from_clause = token.nbor().text
        elif token.dep_ == 'nsubj':
            if token.head.text.lower() in ('is', 'are'):
                select_clause = token.text

    # Search for a simple WHERE clause
    for token in doc:
        if token.dep_ == 'prep' and token.text.lower() == 'where':
            condition = [token.nbor().text, token.nbor().nbor().text, token.nbor().nbor().nbor().text]
            where_clause = " ".join(condition)

    # Formulate SQL
    sql = f"SELECT {select_clause} FROM {from_clause}"
    if where_clause:
        sql += f" WHERE {where_clause}"

    return sql

# Example use with spaCy
nlp = spacy.load("en_core_web_sm")
doc = nlp("How many heads of the departments are older than 56 ?")
sql_query = spacy_nlp_to_sql(doc)
print(sql_query)

SELECT heads FROM []


In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

def allen_nlp_to_sql(doc):
    select_clause = []
    from_clause = []
    where_clause = []
    conditions = []

    # Process each token in the parsed document
    for token in doc['words']:
        idx = doc['words'].index(token)
        pos_tag = doc['pos'][idx]
        dep_rel = doc['predicted_dependencies'][idx]
        head_idx = doc['predicted_heads'][idx] - 1  # Adjusting because AllenNLP uses 1-based indexing
        head_word = doc['words'][head_idx] if head_idx >= 0 else None

        # Simple rule to identify potential SELECT attributes
        if pos_tag.startswith("NN"):  # Simplified POS check
            if token.lower() not in ["number", "count", "total"]:  # avoiding aggregate functions for simplicity
                select_clause.append(token)

        # Detecting potential FROM clause values
        if dep_rel == "nmod" and pos_tag.startswith("NNP"):
            from_clause.append(token)

        # Building WHERE clause based on adjectives or numeric modifiers
        if dep_rel in ["amod", "nummod"] and head_word:
            conditions.append(f"{head_word} = '{token}'")

    # Constructing clauses from identified parts
    if not select_clause:  # default select clause if none identified
        select_clause.append("*")
    select_part = "SELECT " + ", ".join(select_clause)
    from_part = "FROM " + ", ".join(from_clause) if from_clause else "FROM [TABLE]"  # Placeholder if no table identified
    where_part = "WHERE " + " AND ".join(conditions) if conditions else ""

    # Combine parts to form final SQL query
    sql_query = f"{select_part} {from_part} {where_part}"
    return sql_query.strip()

# Example use
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")
sentence = "How many heads of the departments are older than 56 ?"
parsed_output = predictor.predict(sentence=sentence)

sql_query = allen_nlp_to_sql(parsed_output)
print(sql_query)


SELECT * FROM [TABLE] WHERE heads = 'older'


In [ ]:
import spacy
from spacy.matcher import Matcher

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

def extract_conditions(doc):
    # Extracting conditions using simple pattern matching
    matcher = Matcher(nlp.vocab)
    pattern = [{'LOWER': 'older', 'OP': '?'}, {'LIKE_NUM': True}]
    matcher.add("NUMERIC_CONDITION", [pattern])
    matches = matcher(doc)
    conditions = []
    for match_id, start, end in matches:
        span = doc[start:end]
        if span.text.lower().startswith('older'):
            condition = f"age > {span.text.split()[-1]}"
        else:
            condition = f"age = {span.text}"
        conditions.append(condition)
    return " AND ".join(conditions)

def nl_to_sql_universal(query):
    doc = nlp(query)

    # These could be dynamic based on your database schema
    select_clause = "SELECT COUNT(*)"
    from_clause = "FROM employees"
    where_clause = ""

    # Extract conditions from the query
    where_clause = extract_conditions(doc)

    # Construct the SQL query
    if where_clause:
        sql_query = f"{select_clause} {from_clause} WHERE {where_clause}"
    else:
        sql_query = f"{select_clause} {from_clause}"

    return sql_query

# Example usage
query = "How many heads of the departments are older than 56 ??"
sql_query = nl_to_sql_universal(query)
print(sql_query)

SELECT COUNT(*) FROM employees WHERE age = 56


In [ ]:
def weak_supervision(sql_candidates, query):
    def score_sql(sql):
        score = 0
        # Heuristic 1: Reward SQL queries that use keywords likely to be in the correct query
        important_keywords = ['SELECT', 'FROM', 'WHERE', 'JOIN', 'ORDER BY', 'GROUP BY']
        for keyword in important_keywords:
            if keyword in sql:
                score += 1

        # Heuristic 2: Penalize overly complex queries (simplicity bias)
        if sql.count(',') > 3 or len(sql.split()) > 10:
            score -= 1

        # Heuristic 3: Check if query terms appear in the SQL (simple context matching)
        query_terms = query.lower().split()
        sql_terms = sql.lower().split()
        common_terms = set(query_terms).intersection(sql_terms)
        score += len(common_terms)

        return score

    # Score each SQL candidate and select the one with the highest score
    best_sql = max(sql_candidates, key=score_sql)
    return best_sql


In [ ]:
print("SELECT * FROM [TABLE] WHERE heads = 'older'")

SELECT * FROM [TABLE] WHERE heads = 'older'
